In [13]:
# 连接as400
import pyodbc
import pandas as pd
import numpy as np

In [2]:
sql1 = '''
    SELECT biqwnr as secteur, SECT.JLQWNR as SECT_ORIG, BIBKNR as univers, NRLFTB as class_log, 
    bim7nx as contenant , bibhnr as article , NRK4TB as Libelle_Article, BIRID2 as date_creation_ctn, 
    FABACE as zone, bitgc2 as adresse, NRRTNI as dim1, NRRUNI as dim2 , NRRVNI as dim3, PCB.NRRPNI as VOLUME, 
    modele as modele, arnofa as famille, lifam2 as lib_famille, NRK9TB as rayon, ETD.GSZKTZ as ETAPE_DE_VIE, 
    case chauss when 'Y' then 'Y' else 'N' end as chaussant ,  
    ABOGSU as triable, ABOHSU as ATR, UE as ue, pcb as pcb,  (PAL.NRRSNI/PCB) as box_by_pal, BIDYQB as qte_stk, 
    PBP as Passion_brand_product, 
    case blu when 'O' then 'O' else 'N' end as Produit_Bleu , 
    case rfid when 'L' then 'LOG' when 'P' then 'PROD' else 'N' end as ART_RFID 
    FROM newges.mtbirep 
    inner join newges.mrhlrep on bitgc2=hltgc2 
    inner join newges.mrnrrep a on (a.nrbhnr = bibhnr and a.NRRMNI = a.NRRNNI) 
    inner join newges.narten on noart=bibhnr 
    inner join newges.smfarep on FARNN3 =HLPHNX 
    left outer join (select * from newges.soabcpp) as soab on abY1N3=BIBHNR 
    left outer join (select JLQWNR ,JLR2TY from newges.sljlcpp where JLP9NR = 10)     as Sect on ( a.NRLFTB = JLR2TY ) 
    left outer join (select gsbhnr as art1, GSZKTZ as chauss from newges.SPGSREP where GSEBCE = 'CHA') as chaussant on ART1 = BIBHNR 
    left outer join (select gsbhnr as art2, GSZKTZ as PBP from newges.sPGSREP where GSEBCE = 'PBP') as PBPF on ART2 = BIBHNR 
    left outer join (select gsbhnr as art3, GSZKTZ as blu from newges.SPGSREP where GSEBCE = 'BLU') as FicBlu on ART3 = BIBHNR 
    left outer join (select gsbhnr as art4, GSZKTZ as RFID from newges.SPGSREP where GSEBCE = 'RFI') as FicRfi on ART4 = BIBHNR  
    left outer join (select nofam as nofam2, lifam as lifam2 from newges.nfami where nostru = '07' ) as ficfam on nofam2= arnofa 
    left outer join (select i6bhnr as i6art, I6EUNZ as modele from   newges.mti6rep ) as Ficmti6 on i6ART = BIBHNR 
    left outer join  (select nrbhnr , NRRSNI , NRRPNI from newges.mrnrrep where NRFOSY in ('HN' , 'HNDU' )) as PCB on (a.NRBHNR = PCB.NRBHNR) 
    left outer join (SELECT GSBHNR, GSEBCE, GSZKTZ FROM  newges.SPGSREP WHERE GSEBCE = 'ETD' ) as ETD on (ETD.GSBHNR = a.NRBHNR) 
    left outer join  (select nrbhnr , NRRSNI from newges.mrnrrep where NRFOSY ='PL') as PAL on (a.NRBHNR = PAL.NRBHNR) 
    WHERE bidyqb<>0 and bitgc2 between 'A' and 'Z'
    '''

In [1]:
# PIC196
sql2 ='''
    SELECT HVSLN2 as Article, HVFVQA as Qte, HVM9SR as TypeCde ,  HVR3N2 as Livraison , HVR1N2 as Poste_Livraison ,
    HVR5N2 as typeTiers, HVR7N2 as NumTiers, JMM5C2 as NomClient, HVAGD2 as DateLivraison, HVADD2 as DateCreation ,
    HVSWN2 as Univers 
    FROM newges.mrhvrep , commun.MTJMREP 
    WHERE  HVNBSR = 'SC' and HVR5N2= JMP9NR and HVR7N2= JMQUNR  and HVR6N2= JMQWNR ORDER BY HVAGD2
    '''

In [30]:
# PIC042
sql3 = '''
    SELECT HVM9SR as typecde, HVSFN2 as secteur, HVSWN2 as univers, hvsln2 as code, HVR5N2 as type_tiers_dest, 
    HVR7N2 as num_tiers_dest, HVAGD2 as date, sum(hvfvqa) , JoD0NR as log_area 
    FROM newges.mrhvrep 
    left outer join (select * from commun.mtjorep where joE9SS='006') as mtjoeo on HVR5N2=JOP9NR and HVR7N2=JOD2NZ 
    WHERE hvagd2<=20250101 and hvnbsr='10'  and hvsln2 in (select bibhnr from newges.mtbirep where bidyqb<>0                                                                                                                    
    and BIHTNX=0) GROUP BY HVM9SR, HVSFN2, HVSWN2 , HVSLN2, HVR5N2, HVR7N2, HVAGD2, JOD0NR                                                                                                                                                                                                                     

    '''

In [35]:
sql_stk028 = '''
     SELECT mtbi.BITGC2 as adresse,                                   mtbi.BIM7NX as cont, mtbi.BIQWNR as Sector,                      mtbi.BIBHNR as Article, mrnr.nrK4TB as libelle,                  snhm.HMX0N3 as Modele, mtbi.BIDYQB as qte ,                                                             
 mrnr.NRLFTB as Classe, Fourn.Fournisseur as Fournisseur,         mtbi.BIBKNR as Univers_Log, mrnr.NRK9TB as Rayon_mag,            mtbi.BIL0SR as nature_stk, mrnr.NRFPSY as perishable_ddm,       (SELECT L3FCSV FROM newges.sql3rep                                                                       
 WHERE L3HCN4 = (select max(L3HCN4) from newges.sql3rep           where L3HDN4 = mtbi.bibhnr)                                      and L3HDN4 = mtbi.bibhnr)                                        as Rotation,                                                                                            
 spgs.gszktz  as Etape_vie,                                       (CASE WHEN (                                                      select sum(A1DNQB) from newges.mta1rep, newges.mtasrep mtas      where A1H1NX = 2 and A1JFN2 = 4  and A1BHNR =                                                          
 MTBI.BIBHNR and A1H2NX= mtas.ASH2NX                               and mtas.ASM7NX = mtbi.BIM7NX and mtas.ASBHNR = mtbi.BIBHNR      )   IS NULL then 'N' ELSE 'O' END) as En_Cours, mtbi.BIRID2 as  Creation_date  FROM newges.mrnrrep mrnr,                                                                
 newges.spgsrep spgs, newges.snhmcpp snhm,                        newges.mtbirep mtbi                                              left outer join (select mtek.ekbhnr, mtjm.JMM5C2 as Fournisseur  from newges.mterrep mter, newges.mtekrep mtek,                                                          
 commun.mtjmrep mtjm where mtek.ekznnr = mter.erznnr              and mtek.ekznnr in (select max(ekznnr) from newges.mtekrep       where ekbhnr = mtek.ekbhnr and EKBLQA > 0) and mtek.EKBLQA > 0   and mter.ERZ2NR= mtjm.JMP9NR                                                                            
 and mter.ERZ1NR = mtjm.JMQUNR                                    and mter.ERZ0NR= mtjm.JMQWNR   ) as Fourn                        on  (Fourn.ekbhnr = mtbi.bibhnr)                                                                                                                                         
 WHERE mtbi.bibhnr = mrnr.nrbhnr                                  and mrnr.NRRMNI =  mrnr.NRRNNI                                   and mtbi.bibhnr = snhm.HMXZN3                                    and mtbi.bitgc2 >= UCASE('A')                                                                     
 and mtbi.bitgc2 <= UCASE('Z')                              and spgs.gsbhnr = mtbi.bibhnr                                    and spgs.GSEBCE ='ETD'                                           group by mtbi.bitgc2, mtbi.bim7nx, mtbi.biqwnr,                                                         
 mtbi.bibhnr, mrnr.nrk4tb, snhm.HMX0N3, mtbi.bidyqb,              mrnr.NRLFTB, Fourn.Fournisseur, mtbi.bibknr , mrnr.NRK9TB,       mtbi.BIL0SR, mrnr.NRFPSY,                                        (SELECT L3FCSV FROM newges.sql3rep                                                                      
 WHERE L3HCN4 = (select max(L3HCN4) from newges.sql3rep           where L3HDN4 = mtbi.bibhnr)                                      and L3HDN4 = mtbi.bibhnr)                                        , spgs.gszktz, mtbi.BIRID2                                                                              
 order by mtbi.bitgc2, mtbi.bim7nx, mtbi.biqwnr,                  mtbi.bibhnr, mrnr.nrk4tb, snhm.HMX0N3, mtbi.bidyqb,              mrnr.NRLFTB, Fourn.Fournisseur, mtbi.bibknr , mrnr.NRK9TB,       mtbi.BIL0SR,                                                                                            
 (SELECT L3FCSV FROM newges.sql3rep                               WHERE L3HCN4 = (select max(L3HCN4) from newges.sql3rep           where L3HDN4 = mtbi.bibhnr)                                      and L3HDN4 = mtbi.bibhnr), spgs.gszktz, mtbi.BIRID2                                                     

'''

In [38]:
driver='{iSeries Access ODBC Driver}'
system = 'CNDKL02'
uid = 'colis96'
pwd = 'colis96'
cnxn = pyodbc.connect(driver=driver, system=system, uid=uid, pwd=pwd)
cursor=cnxn.cursor()
print('连接成功')
# cursor.execute(sql2)
# columns = [column[0] for column in cursor.description]  
# data = cursor.fetchall()
# df = pd.DataFrame(list(data), columns=columns)
# data

连接成功


In [39]:
d = pd.read_sql(sql_stk028,cnxn)
d

C:\Users\Jinghui\AppData\Local\Temp\ipykernel_30304\4286988883.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  d = pd.read_sql(sql_stk028,cnxn)


,ADRESSE,CONT,SECTOR,ARTICLE,LIBELLE,MODELE,QTE,CLASSE,FOURNISSEUR,UNIVERS_LOG,RAYON_MAG,NATURE_STK,PERISHABLE_DDM,ROTATION,ETAPE_VIE,EN_COURS,CREATION_DATE
0,b'\xc1\xc7\xe5\xf0\xf1\xf1\xf1\xf1',83992.0,96.0,2891152.0,b'\xd7\xc1\xc4\xc4\xc5\xc4@\xc3\xd6\xd4\xc6\xd...,8595380.0,0.0,b'\xc6\xe2\xd4\xe4@',SHANGAI HUB 68,63.0,b'\xf0\xf0\xf6\xf1\xf2',b'@@',b'\xd5',None,b'\xf4@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20220423.0
1,b'\xc1\xc7\xe5\xf0\xf1\xf1\xf1\xf1',349229.0,96.0,4144457.0,b'\xd3\xe2@\xe3\xe2@\xd9\xe4\xd5@\xe6\xc1\xd9\...,8603912.0,0.0,b'\xc6\xe2\xd4\xe4@',SHANGAI HUB 68,46.0,b'\xf0\xf0\xf4\xf0\xf0',b'@@',b'\xd5',None,b'\xf4@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20220423.0
2,b'\xc1\xd4\xc9\xd9\xf0\xf1\xf1\xf1',70185.0,98.0,5660817.0,b'\xd5\xd6\xc2\xc1\xc4\xc1\xe8@\xd4\xc1\xe7@\x...,8966885.0,0.0,b'\xf0\xc8\xd3\xd6@',BEI JING NOBADAY CULTURE CO.,55.0,b'\xf0\xf0\xf1\xf1\xf7',b'@@',b'\xd5',b'\xc3',b'\xf1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20241209.0
3,b'\xc1\xd4\xc9\xd9\xf0\xf1\xf1\xf1',70195.0,98.0,5660772.0,b'\xd5\xd6\xc2\xc1\xc4\xc1\xe8@\xd1\xe4\xc9\xc...,8966877.0,0.0,b'\xf0\xc8\xd3\xd6@',BEI JING NOBADAY CULTURE CO.,57.0,b'\xf0\xf0\xf1\xf1\xf7',b'@@',b'\xd5',b'\xc3',b'\xf1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20241209.0
4,b'\xc1\xd4\xc9\xd9\xf0\xf1\xf1\xf1',70197.0,98.0,5660771.0,b'\xd5\xd6\xc2\xc1\xc4\xc1\xe8@\xd1\xe4\xc9\xc...,8966877.0,0.0,b'\xf0\xc8\xd3\xd6@',BEI JING NOBADAY CULTURE CO.,57.0,b'\xf0\xf0\xf1\xf1\xf7',b'@@',b'\xd5',b'\xc3',b'\xf1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20241209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51789,b'\xe8\xf9\xf9\xd9\xf0\xf1\xf1\xf1',598356.0,96.0,4826164.0,b'\xd1\xc1\xc3\xd2\xc5\xe3@\xd9\xe4\xd5@\xe6\x...,8796834.0,375.0,b'\xc6\xe2\xd4\xe4@',Tianjin Kongang,47.0,b'\xf0\xf0\xf4\xf0\xf0',b'@@',b'\xd5',b'\xc3',b'\xf1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20241227.0
51790,b'\xe8\xf9\xf9\xd9\xf0\xf1\xf1\xf1',598377.0,96.0,4826165.0,b'\xd1\xc1\xc3\xd2\xc5\xe3@\xd9\xe4\xd5@\xe6\x...,8796834.0,101.0,b'\xc6\xe2\xd4\xe4@',Tianjin Kongang,47.0,b'\xf0\xf0\xf4\xf0\xf0',b'@@',b'\xd5',b'\xc3',b'\xf1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20241227.0
51791,b'\xe8\xf9\xf9\xd9\xf0\xf1\xf1\xf1',598381.0,96.0,4826166.0,b'\xd1\xc1\xc3\xd2\xc5\xe3@\xd9\xe4\xd5@\xe6\x...,8796834.0,61.0,b'\xc6\xe2\xd4\xe4@',Tianjin Kongang,47.0,b'\xf0\xf0\xf4\xf0\xf0',b'@@',b'\xd5',b'\xc3',b'\xf1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20241227.0
51792,b'\xe8\xf9\xf9\xd9\xf0\xf1\xf1\xf1',598395.0,96.0,4826167.0,b'\xd1\xc1\xc3\xd2\xc5\xe3@\xd9\xe4\xd5@\xe6\x...,8796834.0,74.0,b'\xc6\xe2\xd4\xe4@',Tianjin Kongang,47.0,b'\xf0\xf0\xf4\xf0\xf0',b'@@',b'\xd5',b'\xc3',b'\xf1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@',N,20241227.0


In [40]:
cnxn.close()
print('连接关闭')

连接关闭
